In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
# from tensorflow import keras
# from keras import layers, optimizers, losses, Model
# from aeon import datasets
import os
# from tqdm import tqdm
import pickle
from input.reading_datasets import get_all_datasets, read_dataset_from_file
import seaborn as sns
from preprocessing.get_dummies_labels import GetDummiesLabels
from preprocessing.train_test_split_module import TrainTestSplit
from models.multi_layer_perceptron import MultiLayerPerceprtron
import shutil

tf.config.list_physical_devices('GPU')

2024-08-02 14:31:20.764616: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-02 14:31:20.775235: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-02 14:31:20.786614: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-02 14:31:20.790520: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-02 14:31:20.800261: I tensorflow/core/platform/cpu_feature_guar

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [15]:
layers_teste = tf.keras.layers.Conv1D(kernel_size= (8,), filters=16)

layers_teste(X[0])

ValueError: Input 0 of layer "conv1d_6" is incompatible with the layer: expected min_ndim=3, found ndim=2. Full shape received: (6, 100)

In [ ]:
all_datasets = [
        "ArticularyWordRecognition", "AtrialFibrillation", "BasicMotions",
        "CharacterTrajectories", "Cricket", "DuckDuckGeese", "EigenWorms",
        "Epilepsy", "EthanolConcentration", "ERing", "FaceDetection",
        "FingerMovements", "HandMovementDirection", "Handwriting", "Heartbeat",
        "JapaneseVowels", "Libras", "LSST", "InsectWingbeat", "MotorImagery",
        "NATOPS", "PenDigits", "PEMS-SF", "Phoneme", "RacketSports",
        "SelfRegulationSCP1", "SelfRegulationSCP2", "SpokenArabicDigits",
        "StandWalkJump", "UWaveGestureLibrary"
    ]

all_datasets.remove('InsectWingbeat')

In [ ]:
# for folder in os.listdir('model_checkpoints'):
#     shutil.rmtree('model_checkpoints/' + folder)

In [17]:
del model

In [25]:
model = tf.keras.models.load_model('/home/samuelthomaz7/fcn_experiment/fully_convolutional_network_mtsc/model_checkpoints/FullyConvolutionalNetwork_articularywordrecognition_1/checkpoint.keras')

ValueError: Expected a model.weights.h5 or model.weights.npz file.

In [22]:
model = tf.keras.models.load_model('model_checkpoints/MultiLayerPerceprtron_basicmotions_1/checkpoint.keras')
dataset = 'BasicMotions'
used_dataset = read_dataset_from_file(dataset_name = dataset)
X, y, metadata = used_dataset

get_dummies_object = GetDummiesLabels(
    X_raw= X,
    y_raw= y,
    metadata= metadata
)

X, y = get_dummies_object.transform()

train_test_object = TrainTestSplit(
    X_raw= X,
    y_raw= y,
    metadata= metadata,
    random_state = 1
)

X_train, X_test, y_train, y_test = train_test_object.transform()
# model.evaluate(X_test, y_test)

In [7]:
model.evaluate(X_test, y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.3125 - f1_score: 0.6000 - loss: 25.7883


[25.78826332092285,
 0.3125,
 <tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.        , 0.59999996, 1.        , 0.79999995], dtype=float32)>]

In [8]:
model.metrics

[<Mean name=loss>, <CompileMetrics name=compile_metrics>]

In [4]:
X_train.shape, X_test.shape

((64, 6, 100), (16, 6, 100))

In [ ]:
X, y, metadata = read_dataset_from_file(dataset_name= 'AtrialFibrillation')

In [24]:
from models.fully_convolutional_network import FullyConvolutionalNetwork
from utils_file import training_nn_for_seeds

training_nn_for_seeds(
    datasets= ['ArticularyWordRecognition'], # type: ignore
    seeds= list(range(1, 2)),
    used_model = FullyConvolutionalNetwork
)

  0%|          | 0/1 [00:00<?, ?it/s]

ArticularyWordRecognition - 1


  0%|          | 0/1 [00:43<?, ?it/s]


KeyboardInterrupt: 